In [4]:
import pandas as pd
import numpy as np
import logging
from math import floor
from google.oauth2.service_account import Credentials
import gspread
from typing import Dict, Any, List
from utils import get_teg_rounds, load_all_data, aggregate_data





In [ ]:
all_data = load_all_data(exclude_teg_50=True,exclude_incomplete_tegs=True)
print(all_data)

In [ ]:
all_data = load_all_data(exclude_teg_50=True,exclude_incomplete_tegs=True)
agg_measures = ['Sc','GrossVP','NetVP','Stableford']
teg_data = aggregate_data(all_data,'TEG',agg_measures)

teg_data

In [ ]:
df = teg_data

best_stableford = df.loc[df.groupby('Player')['Stableford'].idxmax()]

# Select only relevant columns for output
output = best_stableford[['Player', 'Stableford', 'TEG']].rename(columns={'Stableford': 'Best Stableford'})

# Display output
print(output)

In [ ]:

# Find unique combinations of TEG and Year
unique_teg_year = all_data[['TEG', 'Year']].drop_duplicates()

# Merge teg_data with the unique TEG-Year combinations to add the Year column
df_merged = pd.merge(teg_data, unique_teg_year, on='TEG', how='left')

# Define metrics and aggregation functions
metrics = {
    #'lowest Sc': 'Sc',
    'lowest GrossVP': 'GrossVP',
    'lowest NetVP': 'NetVP',
    'highest Stableford': 'Stableford'
}

# Initialize a dictionary to store outputs
output_dict = {}

# Loop over each metric and create the desired DataFrame
for metric_name, column in metrics.items():
    # Set up whether to find max or min for each metric
    if column == 'Stableford':
        best_values = df_merged.loc[df_merged.groupby('Player')[column].idxmax()]
    else:
        best_values = df_merged.loc[df_merged.groupby('Player')[column].idxmin()]

    # Select relevant columns and rename them
    output = best_values[['Player', column, 'TEG', 'Year']].rename(columns={column: metric_name})

    # Store the result in the dictionary
    output_dict[metric_name] = output
    
    # Print the output DataFrame
    print(f"Output for {metric_name}:")
    print(output, "\n")

In [ ]:
aggregate_data(data=all_data,aggregation_level='TEG')

In [ ]:
import pandas as pd
from IPython.display import Markdown

# Example DataFrame
data = {
    'Player': ['Alice', 'Bob', 'Charlie'],
    'Score': [72, 68, 74],
    'TEG': ['TEG1', 'TEG2', 'TEG1'],
    'Year': [2021, 2020, 2022],
    'Measure': ['Gross', 'Net', 'Stableford']
}

df = pd.DataFrame(data)

# Loop through DataFrame and display markdown text with dynamic content
for index, row in df.iterrows():
    # Define the markdown text using row values
    markdown_text = f"Best {row['Measure']}: **{row['Score']}** | {row['Player']} ({row['TEG']}, {row['Year']})"
    
    # Display the markdown text in Jupyter notebook
    display(Markdown(markdown_text))

In [ ]:
from utils import get_round_data, get_complete_teg_data, get_9_data

rds = get_round_data()
tegs = get_complete_teg_data()

#rds.head()

import pandas as pd

rank_field = 'Sc'
rank_ascending = True

def add_ranks(df, fields_to_rank=None, rank_ascending=None):
    # If fields_to_rank is not provided, use default list of fields
    if fields_to_rank is None:
        fields_to_rank = ['Sc', 'GrossVP', 'NetVP', 'Stableford']
    
    # Check if fields_to_rank is a string, convert to list if necessary
    if isinstance(fields_to_rank, str):
        fields_to_rank = [fields_to_rank]
    
    for field in fields_to_rank:
        # Determine default value for rank_ascending for each field
        if rank_ascending is None:
            rank_ascending = False if 'Stableford' in field else True
        
        # Rank within each Player's rounds
        df[f'Rank_within_player_{field}'] = df.groupby('Player')[field].rank(method='min', ascending=rank_ascending)
        
        # Rank across all rounds
        df[f'Rank_within_all_{field}'] = df[field].rank(method='min', ascending=rank_ascending)
    
    return df

#ranked_rds = add_ranks(rds,'Sc',rank_ascending=True)
#ranked_rds.to_clipboard()
# fields_to_rank = 'Sc', ''
# ranked_tegs = add_ranks(tegs,'Stableford')
# ranked_tegs.to_clipboard()

frontback_data = get_9_data()
ranked_9s = add_ranks(df = frontback_data)
ranked_9s.to_clipboard()
print('Copied to clipboard')

In [ ]:
for column in ranked_9s.columns:
    print(column)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# scatter = plt.scatter(df['Rank_within_all_Sc'], df['Sc'])
df = ranked_9s

df['rank'] = df['Rank_within_all_Sc']

# Get unique values from 'Pl' column
unique_pl = df['Pl'].unique()

# Create a color map
color_map = plt.cm.get_cmap('tab10')  # You can change 'tab10' to other colormaps
color_dict = {pl: color_map(i/len(unique_pl)) for i, pl in enumerate(unique_pl)}

# Map colors to your data
colors = df['Pl'].map(color_dict)

plt.figure(figsize=(10, 6))
for pl in unique_pl:
    mask = df['Pl'] == pl
    plt.scatter(df.loc[mask, 'rank'], df.loc[mask, 'Sc'], 
                c=[color_dict[pl]], label=pl)

plt.xlabel('Rank')
plt.ylabel('Mean (Sc)')
plt.title('Scatter Plot of Rank vs Mean (Sc), colored by Pl')
plt.legend()
plt.show()


import plotly.express as px

fig = px.scatter(df, x='rank', y='Sc', color='Pl', 
                 labels={'rank': 'Rank', 'Sc': 'Mean (Sc)', 'Pl': 'Pl'},
                 title='Scatter Plot of Rank vs Mean (Sc), colored by Pl')
fig.show()

In [ ]:
from utils import get_ranked_teg_data, get_ranked_round_data, get_ranked_frontback_data

import streamlit as st

st.cache_data.clear()

tegs_ranked = get_ranked_teg_data()
# for col in tegs_ranked.columns:
#     print(col)

def get_best(df, measure_to_use, player_level = False, top_n = 1):
    valid_measures = ['Sc', 'GrossVP', 'NetVP', 'Stableford']
    if measure_to_use not in valid_measures:
        error_message = f"Invalid measure: '{measure_to_use}'. Valid options are: {', '.join(valid_measures)}"

    if player_level is None:
        player_level = False

    if top_n is None:
        top_n = 1
    
    measure_fn = 'Rank_within_' + ('player' if player_level else 'all') + f'_{measure_to_use}' 

    #measure_fn
    return df[df[measure_fn] == top_n]

#get_best(get_ranked_round_data(), measure_to_use='Stableford')
get_best(tegs_ranked, measure_to_use='Stableford')

In [11]:
import pandas as pd

# Assuming you have your get_best function defined somewhere
# from your_module import get_best

def display_best_records(df):
    measures = ['Sc', 'GrossVP', 'NetVP', 'Stableford']
    
    print("Best Records from Tournaments:")
    print("==============================")
    
    for measure in measures:
        print(f"\nBest {measure}")
        print("-" * (len(measure) + 5))
        
        try:
            best_record = get_best(df, measure_to_use=measure)
            
            # # Create a DataFrame for display
            # display_df = pd.DataFrame({
            #     "Player": [best_record['Player']],
            #     "Score": [best_record[measure]],
            #     "Year": [best_record['Year']],
            #     "TEG": [best_record['TEG']]
            # })
            
            # Display the DataFrame
            #print(display_df.to_string(index=False))
            print(best_record)

            # # Optional: Add more details
            # print(f"\nAchieved in Round {best_record['Round']} of {best_record['TEG']}")
            
        except Exception as e:
            print(f"Error retrieving best {measure}: {str(e)}")
        
        print("\n" + "=" * 40)  # Add a separator between measures

# In your Jupyter Notebook
def main():
    print("Tournament Best Records")
    print("=======================")
    
    # Load your data
    teg_data = get_ranked_teg_data()  # Replace with your data loading function
    
    if teg_data is not None:
        display_best_records(teg_data)
    else:
        print("Failed to load data.")

# Run the main function
main()

2024-10-13 14:00:10.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 14:00:10.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 14:00:10.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 14:00:10.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 14:00:10.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Tournament Best Records
Best Records from Tournaments:

Best Sc
-------
    TEGNum  Year  Pl     TEG        Player    HC     Sc  GrossVP  NetVP  \
39      10  2017  DM  TEG 10  David MULLIN  17.0  337.0     51.0  -17.0   
58      13  2020  JB  TEG 13     Jon BAKER  16.0  337.0     50.0  -14.0   

    Stableford  Rank_within_player_Sc  Rank_within_all_Sc  \
39       162.0                    1.0                 1.0   
58       158.0                    1.0                 1.0   

    Rank_within_player_GrossVP  Rank_within_all_GrossVP  \
39                         1.0                      2.0   
58                         1.0                      1.0   

    Rank_within_player_NetVP  Rank_within_all_NetVP  \
39                       2.0                   10.0   
58                       2.0                   11.0   

    Rank_within_player_Stableford  Rank_within_all_Stableford  
39                            2.0                        11.0  
58                            3.0             